<a href="https://colab.research.google.com/github/arpitamangal/pharma-scrape-and-analysis/blob/main/1mgScrape%26Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from bs4 import BeautifulSoup
import requests
import time
import re
import json
import csv
import pymongo
import pandas as pd
from string import ascii_lowercase as alc
headers={'User-Agent': 'Mozilla/5.0'}

## Scrape Urls for medicines listed alphabetically

In [ ]:
def scrape_medicine_urls():
    try:
        for alphabet in alc:
            url = "https://www.1mg.com/drugs-all-medicines?"
            page = requests.get(url,
                            headers=headers,
                            params={"label": str(alphabet)})
            soup = BeautifulSoup(page.text, 'lxml')
            count_int = 500
            product_per_page = 30
            total_page = count_int/product_per_page
            page_count = round(total_page)
            if (page_count<total_page): page_count = page_count+1
                
            url_list = []
            for index in range(1,(page_count+1)):
                print("iterating through page",index, "of", page_count)
                page_new = requests.get(url,
                            headers=headers,
                            params={"page": str(index),"label": alphabet})
                soup_new = BeautifulSoup(page_new.text, 'lxml')
                list_script = soup_new.find('script', attrs={'type': 'application/ld+json'})
                script_content = list_script.contents[0]
                script_itemList = json.loads(script_content)['itemListElement']

                for item in script_itemList:
                    url_list.append(item["url"])

                print("Length of url_list:", len(url_list))
                index = index+1
                time.sleep(5)

            filename = "1mg_url_list_" + alphabet + ".csv"
            pd.DataFrame(url_list).to_csv(filename,index=False)
            print("Saved URLS for medicine names starting with:",alphabet)
            print("")
            time.sleep(5)
            
    except:
        print("Error while scraping medicine URLS...")
            

## Import CSV and get the url list 

In [ ]:
def url_list_gen (alphabet):
    try:
        url_list = []
        file_path = f'1mg_url_list_{alphabet}.csv'

        with open(file_path, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            for row in csvreader:
                url_list.append(row[0])
        return url_list
    except:
        print("Error while reading CSV files...")

## Download HTML files for each medicine

In [ ]:
def download_html_pages():
    try: 
        for alphabet in alc:  # loop over first 4 alphabets
            index = 0
            url_list = url_list_gen(alphabet)
            for url in url_list:
                if index == 0:
                    index += 1 
                    continue
                headers = {'User-Agent': 'Mozilla/5.0'}
                response = requests.get(url, headers=headers)
                soup = BeautifulSoup(response.text, 'lxml')
                fp = f"1mg_{alphabet}{index}.htm"
                with open(fp, "w", encoding="utf-8") as file:
                    file.write(str(soup))
                index += 1 
                if index == 500:
                    break
        print("Saved HTML files")
    except: 
        print("Error while downloading HTML pages...")

## Scarp details of medicine from HTML file

In [ ]:
def get_elements(soup):
    try:    
        div_elements = soup.select('div.flexColumn')
        dict_med = {}

        info_json_list = soup.find_all('script', attrs={'type': 'application/ld+json'})
        for element in info_json_list:
            if "manufacturer" in str(element):
                info_json = json.loads(element.contents[0])
                dict_med['id'] = re.search(r".+?([0-9]{4}[0-9]*)", info_json['url']).group(1)
                print(f"id :{dict_med['id']}") 
                dict_med['manufacturer'] = info_json['manufacturer']
                dict_med['nonProprietaryName'] = info_json['nonProprietaryName']
                dict_med['dosageForm'] = info_json['dosageForm']
                dict_med['activeIngredient'] = info_json['activeIngredient']
                dict_med['mechanismOfAction'] = info_json['mechanismOfAction']
                dict_med['interactingDrug'] = info_json['interactingDrug']
            
        if div_elements:
            med = div_elements[0]

            name_ele = med.select_one("div.col-6.marginTop-8.flex.alignFlexEnd > div > div > h1")
            name = name_ele.text.strip() if name_ele else print("Not found")
#             print(f"Name :{name}")
            dict_med["Name"] = name
            
            price_ele = med.select_one("span.l4Regular.PriceWidget__marginLeft__dk5gl.PriceWidget__strikeThrough__rJY6f")
            price = price_ele.text.strip() if price_ele else print("Not found")
            if price is not None:
                price = str(price)
                match = re.search(r'MRP ₹(.+)', price)
                if match:
                    price = match.group(1)
#                     print(f"Price: {price}")
                    dict_med["Price"] = price
                else:
#                     print("Price not found")
                    dict_med["Price"] =  " Not found "
            else:
#                 print("Not found")
                dict_med["Price"] =  " Not found "

            desc_ele = med.select_one("div.col-6.bodyRegular.textPrimary.marginTop-8")
            desc = desc_ele.text.strip() if desc_ele else print("Not found")
#             print(f"Description :{desc}")
#             print()
            dict_med["Description"] = desc

            pro_ele = med.select_one("div:nth-child(27) > div.col-6.marginTop-8.GeneralDescription__htmlNodeWrapper__h23K3")

            pro = pro_ele.text.strip() if pro_ele else print("Not found")
#             print(f"Product Info :{pro}")
#             print()
            dict_med["Product Info"] = pro

            use_ele = med.select_one("div.col-6.marginTop-8.GeneralDescription__htmlNodeWrapper__h23K3 > ul > li > a")
            use = use_ele.text.strip() if use_ele else print("Not found")
#             print(f"Used for :{use}")
#             print()
            dict_med["Uses"] = use

            bene_ele = med.select_one("div.col-6.marginTop-8.GeneralDescription__htmlNodeWrapper__h23K3 > div")
            bene = bene_ele.text.strip() if bene_ele else print("Not found")
#             print(f"Benefits :{bene}")
#             print()
            dict_med["Benefits"] = bene

            side_ele = med.select("div.col-6.marginTop-8.GeneralDescription__htmlNodeWrapper__h23K3 > div > ul > li")
            side_list = [side.text for side in side_ele]
#             print(f"Side Effects :{side_list}")
#             print()
            dict_med["Side Effects"] = side_list

            work_ele = med.select_one("div:nth-child(37) > div.col-6.marginTop-8.GeneralDescription__htmlNodeWrapper__h23K3")
            work = work_ele.text.strip() if work_ele else print("Not found")
#             print(f"Working :{work}")
#             print()
            dict_med["Workings"] = work

        match = re.search(r"factBoxData\":(.+?),\"userResponses", str(soup))
        if match:
            factBoxData = match.group(1)
            factboxDict = {}
            for label in json.loads(factBoxData)['attributesData']:
                factboxDict[label['ga_label']] = label['value']   
#             print(f"Factbox: {factboxDict}")
            dict_med['Factbox'] = factboxDict

        match = re.search(r"productSubstitutes\":(.+?),\"productHighLight", str(soup))
        if match:    
            productSubstitutes = match.group(1)
            if (productSubstitutes != "null"):
                productSubList = []
                for label in json.loads(productSubstitutes)['attributesData']:
                    productSubList.append({"Name":BeautifulSoup(label['header'],"html.parser").text,"id":label['id']})
#                 print(f"Substitute: {productSubList}")
                dict_med['Substitute'] = productSubList

        print("-----------------")
        return dict_med
    except:
        print("Error while scraping details of medicine...")

## Call function to scrap the medicine details and Store in MongoDB

In [ ]:
def get_elements_super():
    try:
        list_med =[]
        
        for alphabet in alc:
            for i in range(1, 500):
                filename = f"_{alphabet}{i}.htm"
                with open(filename, "r", encoding="utf-8") as file:
                    html = file.read() 
                soup = BeautifulSoup(html, "lxml")
                print()
                print(f"Scraping elements from file: {filename}")
                med_dict = get_elements(soup)
                if bool(med_dict):
                    list_med.append(med_dict)
        print("Stored data as a list of dictionaries")
        return list_med
    except: 
        print("Error while scraping html pages...")

## Store Data in MongoDB

In [ ]:
def store_in_mongodb(med_list):
    try:
        client = pymongo.MongoClient("mongodb://localhost:27017")
        database = client["Medicines"]
        collection = database["Medicines"]
        for med in med_list:
            collection.insert_one(med)
        print("Stored data in MongDB")
    except: 
        print("Error while storing data in Mongodb...")

In [ ]:
##scrape medicine urls
scrape_medicine_urls()
##download html for each medicine
download_html_pages()
##store the information as list of dictonary 
med_list = get_elements_super()
##store the data on monodb
store_in_mongodb(med_list)


Scraping elements from file: _a1.htm
id :138629
-----------------

Scraping elements from file: _a2.htm
id :63412
-----------------

Scraping elements from file: _a3.htm
id :325616
-----------------

Scraping elements from file: _a4.htm
id :68763
Not found
-----------------

Scraping elements from file: _a5.htm
id :491614
-----------------

Scraping elements from file: _a6.htm
id :69629
-----------------

Scraping elements from file: _a7.htm
id :324155
Not found
-----------------

Scraping elements from file: _a8.htm
id :184611
-----------------

Scraping elements from file: _a9.htm
id :9172
Not found
-----------------

Scraping elements from file: _a10.htm
id :63175
-----------------

Scraping elements from file: _a11.htm
id :134660
-----------------

Scraping elements from file: _a12.htm
id :228160
-----------------

Scraping elements from file: _a13.htm
id :324979
-----------------

Scraping elements from file: _a14.htm
id :543127
Not found
-----------------

Scraping elements from

id :624176
-----------------

Scraping elements from file: _a118.htm
id :17666
-----------------

Scraping elements from file: _a119.htm
id :123250
-----------------

Scraping elements from file: _a120.htm
id :63274
Not found
-----------------

Scraping elements from file: _a121.htm
id :333029
-----------------

Scraping elements from file: _a122.htm
id :141365
-----------------

Scraping elements from file: _a123.htm
id :513352
-----------------

Scraping elements from file: _a124.htm
id :158993
Not found
-----------------

Scraping elements from file: _a125.htm
id :58521
-----------------

Scraping elements from file: _a126.htm
id :67683
-----------------

Scraping elements from file: _a127.htm
id :68677
-----------------

Scraping elements from file: _a128.htm
id :63311
-----------------

Scraping elements from file: _a129.htm
id :63478
Not found
-----------------

Scraping elements from file: _a130.htm
id :678845
-----------------

Scraping elements from file: _a131.htm
id :43925
-

-----------------

Scraping elements from file: _a232.htm
id :196931
-----------------

Scraping elements from file: _a233.htm
id :389182
Not found
-----------------

Scraping elements from file: _a234.htm
id :385132
-----------------

Scraping elements from file: _a235.htm
id :372596
Not found
Not found
-----------------

Scraping elements from file: _a236.htm
id :58520
-----------------

Scraping elements from file: _a237.htm
id :770655
-----------------

Scraping elements from file: _a238.htm
id :158246
-----------------

Scraping elements from file: _a239.htm
id :335338
-----------------

Scraping elements from file: _a240.htm
id :655405
Not found
Not found
-----------------

Scraping elements from file: _a241.htm
id :530952
-----------------

Scraping elements from file: _a242.htm
id :340528
-----------------

Scraping elements from file: _a243.htm
id :333157
-----------------

Scraping elements from file: _a244.htm
id :182339
Not found
-----------------

Scraping elements from fi


Scraping elements from file: _a345.htm
id :338069
-----------------

Scraping elements from file: _a346.htm
id :275872
Not found
-----------------

Scraping elements from file: _a347.htm
id :43985
-----------------

Scraping elements from file: _a348.htm
id :157514
-----------------

Scraping elements from file: _a349.htm
id :644803
Not found
-----------------

Scraping elements from file: _a350.htm
id :339728
-----------------

Scraping elements from file: _a351.htm
id :314338
-----------------

Scraping elements from file: _a352.htm
id :146602
-----------------

Scraping elements from file: _a353.htm
id :134307
-----------------

Scraping elements from file: _a354.htm
id :116801
-----------------

Scraping elements from file: _a355.htm
id :58367
Not found
-----------------

Scraping elements from file: _a356.htm
id :792551
Not found
-----------------

Scraping elements from file: _a357.htm
id :140441
-----------------

Scraping elements from file: _a358.htm
id :150814
--------------

-----------------

Scraping elements from file: _a459.htm
id :504295
-----------------

Scraping elements from file: _a460.htm
id :277251
-----------------

Scraping elements from file: _a461.htm
id :636852
Not found
Not found
-----------------

Scraping elements from file: _a462.htm
id :39511
Not found
-----------------

Scraping elements from file: _a463.htm
id :582295
Not found
-----------------

Scraping elements from file: _a464.htm
id :660034
Not found
-----------------

Scraping elements from file: _a465.htm
id :260587
-----------------

Scraping elements from file: _a466.htm
id :58435
-----------------

Scraping elements from file: _a467.htm
id :162893
-----------------

Scraping elements from file: _a468.htm
id :545307
Not found
-----------------

Scraping elements from file: _a469.htm
id :453732
Not found
-----------------

Scraping elements from file: _a470.htm
id :148773
-----------------

Scraping elements from file: _a471.htm
id :125015
-----------------

Scraping element

...

-----------------

Scraping elements from file: _y447.htm
id :804314
Not found
-----------------

Scraping elements from file: _y448.htm
id :759359
-----------------

Scraping elements from file: _y449.htm
id :413928
Not found
-----------------

Scraping elements from file: _y450.htm
id :229267
Not found
-----------------

Scraping elements from file: _y451.htm
id :532138
Not found
-----------------

Scraping elements from file: _y452.htm
id :589533
Not found
Not found
-----------------

Scraping elements from file: _y453.htm
id :614352
Not found
Not found
-----------------

Scraping elements from file: _y454.htm
id :612694
Not found
-----------------

Scraping elements from file: _y455.htm
id :598760
Not found
-----------------

Scraping elements from file: _y456.htm
id :674083
-----------------

Scraping elements from file: _y457.htm
id :759366
Not found
-----------------

Scraping elements from file: _y458.htm
id :691195
Not found
Not found
-----------------

Scraping elements from 

-----------------

Scraping elements from file: _z59.htm
id :546496
Not found
-----------------

Scraping elements from file: _z60.htm
id :41887
-----------------

Scraping elements from file: _z61.htm
id :32456
-----------------

Scraping elements from file: _z62.htm
id :53185
-----------------

Scraping elements from file: _z63.htm
id :67401
-----------------

Scraping elements from file: _z64.htm
id :332091
Not found
-----------------

Scraping elements from file: _z65.htm
id :740497
-----------------

Scraping elements from file: _z66.htm
id :9668
-----------------

Scraping elements from file: _z67.htm
id :482699
-----------------

Scraping elements from file: _z68.htm
id :18426
Not found
-----------------

Scraping elements from file: _z69.htm
id :565879
-----------------

Scraping elements from file: _z70.htm
id :518323
-----------------

Scraping elements from file: _z71.htm
id :71334
-----------------

Scraping elements from file: _z72.htm
id :137991
Not found
----------------

-----------------

Scraping elements from file: _z174.htm
id :150244
Not found
Not found
-----------------

Scraping elements from file: _z175.htm
id :485650
-----------------

Scraping elements from file: _z176.htm
id :32483
-----------------

Scraping elements from file: _z177.htm
id :72990
-----------------

Scraping elements from file: _z178.htm
id :341192
-----------------

Scraping elements from file: _z179.htm
id :304401
Not found
-----------------

Scraping elements from file: _z180.htm
id :254008
-----------------

Scraping elements from file: _z181.htm
id :370701
-----------------

Scraping elements from file: _z182.htm
id :268695
-----------------

Scraping elements from file: _z183.htm
id :74723
Not found
-----------------

Scraping elements from file: _z184.htm
id :240459
-----------------

Scraping elements from file: _z185.htm
id :315632
-----------------

Scraping elements from file: _z186.htm
id :123278
-----------------

Scraping elements from file: _z187.htm
id :2452

Not found
-----------------

Scraping elements from file: _z288.htm
id :60482
-----------------

Scraping elements from file: _z289.htm
id :677991
-----------------

Scraping elements from file: _z290.htm
id :497684
-----------------

Scraping elements from file: _z291.htm
id :157507
-----------------

Scraping elements from file: _z292.htm
id :25451
Not found
-----------------

Scraping elements from file: _z293.htm
id :114442
-----------------

Scraping elements from file: _z294.htm
id :283665
-----------------

Scraping elements from file: _z295.htm
id :782521
Not found
-----------------

Scraping elements from file: _z296.htm
id :595855
-----------------

Scraping elements from file: _z297.htm
id :565919
-----------------

Scraping elements from file: _z298.htm
id :140728
-----------------

Scraping elements from file: _z299.htm
id :253074
-----------------

Scraping elements from file: _z300.htm
id :701182
Not found
-----------------

Scraping elements from file: _z301.htm
id :561

-----------------

Scraping elements from file: _z401.htm
id :306548
-----------------

Scraping elements from file: _z402.htm
id :341815
-----------------

Scraping elements from file: _z403.htm
id :60460
-----------------

Scraping elements from file: _z404.htm
id :370202
Not found
-----------------

Scraping elements from file: _z405.htm
id :131309
-----------------

Scraping elements from file: _z406.htm
Error while scraping details of medicine...

Scraping elements from file: _z407.htm
id :494631
-----------------

Scraping elements from file: _z408.htm
id :341117
Not found
-----------------

Scraping elements from file: _z409.htm
id :232032
Not found
-----------------

Scraping elements from file: _z410.htm
id :233940
-----------------

Scraping elements from file: _z411.htm
id :164765
-----------------

Scraping elements from file: _z412.htm
id :840969
Not found
-----------------

Scraping elements from file: _z413.htm
id :290832
-----------------

Scraping elements from file: _

## Save the data as a CSV for analysis

In [ ]:
import pandas as pd
pd.DataFrame(med_list).to_csv("1mg_medicines.csv",index=False)